# Detecção de Fraudes em Transações de Cartão de Crédito

Este notebook implementa um pipeline de Machine Learning de ponta a ponta para detectar transações fraudulentas de cartão de crédito.
Ele inclui:
1.  Carregamento de dados (reais ou geração de dados sintéticos).
2.  Engenharia de features para melhorar a capacidade preditiva do modelo.
3.  Técnicas para lidar com datasets desbalanceados (SMOTE).
4.  Divisão dos dados em conjuntos de treino e teste.
5.  Treinamento de um modelo `RandomForestClassifier`.
6.  Avaliação do modelo utilizando diversas métricas e análise da matriz de confusão.
7.  Visualização da importância das features.

## 1. Importações de Bibliotecas

Este bloco de código Python é dedicado à **importação das bibliotecas e módulos necessários** para o projeto. Ele carrega `pandas` e `numpy` para manipulação de dados, `matplotlib` e `seaborn` para visualização, e diversos componentes do `scikit-learn` para geração de dados sintéticos (`make_classification`), divisão de dados (`train_test_split`), modelagem (`RandomForestClassifier`) e avaliação (`classification_report`, `confusion_matrix`, `accuracy_score`). Crucialmente, tenta importar `SMOTE` da biblioteca `imbalanced-learn`, definindo `IMBLEARN_AVAILABLE` para indicar se o oversampling com SMOTE estará funcional.


In [ ]:
# Manipulação de dados e matemática
import pandas as pd
import numpy as np
import os
import time

# Visualização 
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn: geração de dados, divisão, modelo, métricas
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Imbalanced-learn: SMOTE para oversampling
# Certifique-se de ter instalado: pip install imbalanced-learn matplotlib seaborn
try:
    from imblearn.over_sampling import SMOTE
    IMBLEARN_AVAILABLE = True
    print("Biblioteca 'imbalanced-learn' (para SMOTE) carregada com sucesso.")
except ImportError:
    IMBLEARN_AVAILABLE = False
    print("AVISO: Biblioteca 'imbalanced-learn' não encontrada. A funcionalidade SMOTE não estará disponível.")
    print("Para instalá-la, execute no seu terminal (com o venv ativo): pip install imbalanced-learn")

# Configurações de estilo para plots (opcional)
# %matplotlib inline # Se estiver usando ambiente Jupyter clássico ou quiser garantir plots inline
# plt.style.use('seaborn-v0_8-whitegrid') # Estilo de plot
# sns.set_style('whitegrid')

print("Bibliotecas e módulos básicos importados.")

## 2. Configuração do Experimento

Este bloco Python define parâmetros cruciais para um experimento de machine learning. Ele permite escolher entre dados reais (`creditcard.csv`) ou sintéticos, habilitar/desabilitar a técnica de oversampling SMOTE no treino, e configurar hiperparâmetros para um modelo `RandomForestClassifier` (como `n_estimators`, `max_depth`). Além disso, estabelece um limiar de classificação (`0.5`) para converter probabilidades em classes e uma semente aleatória (`42`) para reprodutibilidade. Ao final, imprime as configurações selecionadas.


In [ ]:
# --- CONFIGURAÇÕES DO EXPERIMENTO ---

# Escolha da Fonte de Dados
USAR_DADOS_REAIS = True  # Mude para False para usar dados sintéticos

CAMINHO_DADOS_REAIS = 'creditcard.csv'
COLUNA_ALVO_REAL = 'Class'

# Configurações para Dados Sintéticos (se USAR_DADOS_REAIS = False)
N_AMOSTRAS_SINTETICOS = 20000
N_FEATURES_SINTETICOS = 20 # Número de features antes da engenharia
PESOS_CLASSES_SINTETICOS = [0.99, 0.01] # Simula desbalanceamento (1% classe positiva)

# Configuração do SMOTE (aplicado ao conjunto de treino)
APLICAR_SMOTE_NO_TREINO = True # Mude para False para não aplicar SMOTE

# Parâmetros do Modelo RandomForestClassifier
# Estes são os parâmetros que você pode querer ajustar baseados nos seus testes com ui.py ou GridSearchCV
MODEL_PARAMS = {
    'n_estimators': 150,
    'max_depth': 30,
    'min_samples_leaf': 1,
    'min_samples_split': 5,
    'random_state': 42,
    'verbose': 0,          # Mantenha 0 para menos output no notebook, 1 ou mais para progresso do RF
    'n_jobs': -1           # Usar todos os cores da CPU
}

# Limiar de Classificação para converter probabilidades em predições de classe
LIMIAR_CLASSIFICACAO = 0.5

# Semente aleatória para reprodutibilidade geral
RANDOM_STATE = 42

print("Configurações do experimento definidas.")
if USAR_DADOS_REAIS:
    print(f"  Modo: Dados Reais ('{CAMINHO_DADOS_REAIS}')")
else:
    print("  Modo: Dados Sintéticos")
print(f"  Aplicar SMOTE no treino: {'Sim' if APLICAR_SMOTE_NO_TREINO else 'Não'}")
print(f"  Parâmetros do Modelo: {MODEL_PARAMS}")
print(f"  Limiar de Classificação: {LIMIAR_CLASSIFICACAO}")

## 3. Funções Utilitárias para Dados

Este bloco de código define **quatro funções Python essenciais** para um pipeline de machine learning. A função `load_csv_data` carrega dados de um arquivo CSV e separa features (X) do alvo (y). `generate_synthetic_data_scratch` cria dados sintéticos para classificação binária usando `make_classification`. `engineer_features_from_data` gera novas features a partir de um DataFrame existente (ex: `Amount_per_Time`, `Log1p_Amount`, features cíclicas de tempo). Por fim, `augment_data_smote` aplica a técnica SMOTE para lidar com desbalanceamento de classes nos dados de treino, se a biblioteca `imblearn` estiver disponível.

In [ ]:
def load_csv_data(file_path, target_column_name):
    """
    Carrega dados de um arquivo CSV especificado e separa as features (X) e o alvo (y).
    """
    print(f"\nTentando carregar dados de: {file_path}...")
    try:
        df = pd.read_csv(file_path)
        print(f"Dados carregados com sucesso de {file_path}.")
        if target_column_name in df.columns:
            X = df.drop(target_column_name, axis=1)
            y = df[target_column_name]
            print(f"Features (X) e alvo (y: '{target_column_name}') separados.")
            print(f"Shape das Features (X): {X.shape}, Shape do Alvo (y): {y.shape}")
            return X, y
        else:
            print(f"Erro: Coluna alvo '{target_column_name}' não encontrada em {file_path}.")
            print(f"Colunas disponíveis: {df.columns.tolist()}")
            return None, None
    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado em {file_path}. Verifique se o caminho está correto.")
        return None, None
    except Exception as e:
        print(f"Ocorreu um erro ao carregar ou processar os dados: {e}")
        return None, None

def generate_synthetic_data_scratch(n_samples, n_features, class_weights, target_column_name, random_state):
    """
    Gera um conjunto de dados sintético para classificação binária a partir do zero.
    """
    print("\nGerando dados sintéticos do zero...")
    # Ajuste n_informative e n_redundant para serem sempre válidos em relação a n_features
    n_informative_actual = max(1, min(n_features -1, int(n_features * 0.75))) 
    if n_features <= n_informative_actual : # Make sure n_features is greater than n_informative
        n_informative_actual = max(1, n_features-1) if n_features > 1 else 1


    n_redundant_actual = max(0, n_features - n_informative_actual - 1)
    if n_features == 1 and n_informative_actual == 1: # Edge case for single feature
        n_redundant_actual = 0


    X_synth, y_synth = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=n_informative_actual,
        n_redundant=n_redundant_actual,
        n_repeated=0,
        n_classes=2,
        n_clusters_per_class=1,
        weights=class_weights,
        flip_y=0.01,
        random_state=random_state
    )
    feature_names = [f'synthetic_feature_{i+1}' for i in range(X_synth.shape[1])]
    X_df = pd.DataFrame(X_synth, columns=feature_names)
    y_series = pd.Series(y_synth, name=target_column_name)
    print(f"Dados sintéticos gerados com shape X: {X_df.shape}, shape y: {y_series.shape}")
    class_dist = y_series.value_counts(normalize=True) * 100
    print(f"Distribuição das classes: \nClasse 0: {class_dist.get(0, 0):.2f}%\nClasse 1: {class_dist.get(1, 0):.2f}%")
    return X_df, y_series

def engineer_features_from_data(X_input_df):
    """
    Cria novas features a partir de um DataFrame de features existente.
    """
    if X_input_df is None:
        print("DataFrame de entrada para engenharia de features é None. Pulando.")
        return None
    print("\nRealizando engenharia de novas features a partir dos dados de entrada...")
    X_engineered = X_input_df.copy()
    if 'Time' in X_engineered.columns and 'Amount' in X_engineered.columns:
        X_engineered['Amount_per_Time'] = X_engineered['Amount'] / (X_engineered['Time'] + 1e-6)
        print("  Feature criada: 'Amount_per_Time'")
    if 'Amount' in X_engineered.columns:
        X_engineered['Log1p_Amount'] = np.log1p(X_engineered['Amount'])
        print("  Feature criada: 'Log1p_Amount'")
    if 'V1' in X_engineered.columns and 'V2' in X_engineered.columns: # Example
        X_engineered['V1_x_V2'] = X_engineered['V1'] * X_engineered['V2']
        print("  Feature criada: 'V1_x_V2'")
    if 'Time' in X_engineered.columns:
        print("  Criando features cíclicas de tempo (Hora do Dia)...")
        seconds_in_day = 24 * 60 * 60
        X_engineered['HourOfDay'] = (X_engineered['Time'] % seconds_in_day) / 3600.0
        X_engineered['HourOfDay_sin'] = np.sin(2 * np.pi * X_engineered['HourOfDay'] / 24.0)
        X_engineered['HourOfDay_cos'] = np.cos(2 * np.pi * X_engineered['HourOfDay'] / 24.0)
        X_engineered = X_engineered.drop('HourOfDay', axis=1)
        print("  Features criadas: 'HourOfDay_sin', 'HourOfDay_cos'")
    print(f"Engenharia de features completa. Novo shape X: {X_engineered.shape}")
    return X_engineered

def augment_data_smote(X_input_df, y_input_series, random_state):
    """
    Aumenta os dados usando SMOTE para lidar com o desbalanceamento de classes.
    """
    if not IMBLEARN_AVAILABLE:
        print("SMOTE requer imbalanced-learn. Retornando dados originais.")
        return X_input_df, y_input_series
    if X_input_df is None or y_input_series is None:
        print("X ou y de entrada para SMOTE é None. Retornando dados originais.")
        return X_input_df, y_input_series
    print("\nTentando aumentar os dados usando SMOTE...")
    try:
        print("Distribuição de classes original no treino:\n", y_input_series.value_counts(normalize=True) * 100)
        if len(y_input_series.value_counts()) < 2 or y_input_series.value_counts().min() < 2: # SMOTE needs min samples
            print("Não há amostras suficientes na classe minoritária ou apenas uma classe presente. Pulando SMOTE.")
            return X_input_df, y_input_series
        smote = SMOTE(random_state=random_state)
        X_smote, y_smote = smote.fit_resample(X_input_df, y_input_series)
        X_smote_df = pd.DataFrame(X_smote, columns=X_input_df.columns)
        y_smote_series = pd.Series(y_smote, name=y_input_series.name)
        print("SMOTE aplicado com sucesso ao conjunto de treino.")
        print(f"Shape X após SMOTE: {X_smote_df.shape}, shape y após SMOTE: {y_smote_series.shape}")
        print("Nova distribuição de classes após SMOTE:\n", y_smote_series.value_counts(normalize=True) * 100)
        return X_smote_df, y_smote_series
    except Exception as e:
        print(f"Erro durante o SMOTE: {e}. Retornando dados originais.")
        return X_input_df, y_input_series

print("Funções utilitárias de dados definidas.")

## 4. Execução do Pipeline Principal

Agora vamos executar o pipeline passo a passo, utilizando as configurações e funções definidas acima.

### 4.1 Carregamento de Dados e Engenharia de Features

Este bloco de código é responsável por **carregar ou gerar o conjunto de dados principal** (`X_final`, `y_final`). Se a configuração `USAR_DADOS_REAIS` for `True`, ele utiliza a função `load_csv_data` para carregar dados reais e, em seguida, aplica `engineer_features_from_data` para criar novas features. Caso contrário (dados sintéticos), ele chama `generate_synthetic_data_scratch`. Ao final, se os dados foram obtidos com sucesso, exibe informações como as dimensões, a distribuição da variável alvo e as primeiras linhas do DataFrame `X_final`, preparando-o para as próximas etapas.

In [ ]:
X_final, y_final = None, None

if USAR_DADOS_REAIS:
    X_initial, y_initial = load_csv_data(file_path=CAMINHO_DADOS_REAIS, target_column_name=COLUNA_ALVO_REAL)
    if X_initial is not None:
        X_final = engineer_features_from_data(X_initial)
        y_final = y_initial # y não muda com a engenharia de features de X
else:
    X_final, y_final = generate_synthetic_data_scratch(
        n_samples=N_AMOSTRAS_SINTETICOS,
        n_features=N_FEATURES_SINTETICOS,
        class_weights=PESOS_CLASSES_SINTETICOS,
        target_column_name=COLUNA_ALVO_REAL, # Usando o mesmo nome de alvo para consistência
        random_state=RANDOM_STATE
    )
    print("(Para dados sintéticos, a etapa de 'engineer_features_from_data' é opcional e dependeria dos nomes das features geradas)")

if X_final is not None and y_final is not None:
    print("\nDados prontos para a próxima etapa.")
    print(f"Shape de X_final: {X_final.shape}")
    print(f"Distribuição de y_final:\n{y_final.value_counts(normalize=True)}")
    print("\nPrimeiras 5 linhas de X_final:")
    display(X_final.head()) # Use display() para melhor formatação de DataFrames em Jupyter
else:
    print("ERRO: Falha no carregamento ou geração de dados. Não é possível continuar.")

### 4.2 Divisão em Treino e Teste

Este trecho de código é responsável por **dividir o conjunto de dados final** (`X_final`, `y_final`) em subconjuntos de **treinamento e teste**. Primeiramente, ele verifica se os dados finais existem. Se sim, utiliza a função `train_test_split` para separar 25% dos dados para teste (`test_size=0.25`), mantendo a proporção das classes (`stratify=y_final`) e usando uma semente aleatória (`RANDOM_STATE`) para reprodutibilidade. Finalmente, exibe as dimensões e a distribuição da variável alvo nos conjuntos resultantes ou uma mensagem de erro/aviso se a divisão não puder ocorrer.

In [ ]:
X_train, X_test, y_train, y_test = None, None, None, None

if X_final is not None and y_final is not None:
    print("\nDividindo os dados em conjuntos de treino e teste...")
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X_final, y_final,
            test_size=0.25,
            random_state=RANDOM_STATE,
            stratify=y_final
        )
        print("Dados divididos com sucesso.")
        print(f"  Shape de X_train: {X_train.shape}, Shape de y_train: {y_train.shape}")
        print(f"  Shape de X_test: {X_test.shape}, Shape de y_test: {y_test.shape}")
        print(f"  Distribuição do alvo no treino original (y_train):\n{y_train.value_counts(normalize=True)}")
        print(f"  Distribuição do alvo no teste (y_test):\n{y_test.value_counts(normalize=True)}")
    except Exception as e:
        print(f"Erro ao dividir os dados: {e}")
else:
    print("Dados finais (X_final, y_final) não estão disponíveis para divisão.")

### 4.3 Aplicação de SMOTE (Opcional, no Conjunto de Treino)

Este bloco de código **prepara os dados de treino** (`X_train`, `y_train`) para o modelo, com a **opção de aplicar a técnica SMOTE**. Inicialmente, `X_train_processed` e `y_train_processed` são criados como cópias dos dados de treino. Se a variável `APLICAR_SMOTE_NO_TREINO` for `True` e os dados de treino existirem, a função `augment_data_smote` é chamada para balancear as classes. Caso contrário, ou se SMOTE não for aplicado, os dados de treino originais são mantidos, com mensagens informativas sobre a ação tomada e a distribuição das classes.


In [ ]:
X_train_processed = X_train.copy() if X_train is not None else None
y_train_processed = y_train.copy() if y_train is not None else None

if X_train is not None and y_train is not None:
    if APLICAR_SMOTE_NO_TREINO:
        print("\nAplicando SMOTE apenas ao conjunto de treino...")
        X_train_processed, y_train_processed = augment_data_smote(X_train, y_train, random_state=RANDOM_STATE)
        # A função augment_data_smote já imprime os shapes e distribuições
    else:
        print("\nSMOTE não aplicado ao conjunto de treino.")
        if X_train_processed is not None: # Apenas para printar se os dados existem
             print(f"Usando dados de treino originais: X_train_processed shape: {X_train_processed.shape}")
             print(f"Distribuição de y_train_processed:\n{y_train_processed.value_counts(normalize=True)}")
else:
    print("Conjunto de treino não disponível. Pulando SMOTE.")

### 4.4 Treinamento do Modelo

Este bloco de código foca no **treinamento do modelo `RandomForestClassifier`**. Antes de treinar, ele ajusta os parâmetros do modelo (`MODEL_PARAMS`): se SMOTE não foi aplicado e `class_weight` não está definido, ele adiciona `class_weight='balanced'`; se SMOTE foi aplicado e `class_weight` é 'balanced', ele o remove para evitar redundância. Em seguida, instancia o `RandomForestClassifier` com os parâmetros ajustados e o treina usando os dados `X_train_processed` e `y_train_processed`, cronometrando a duração do processo. Tratamento de erros e mensagens de progresso/sucesso são incluídos.

In [ ]:
model = None
training_duration = 0

if X_train_processed is not None and y_train_processed is not None:
    current_model_params_for_training = MODEL_PARAMS.copy()
    if not APLICAR_SMOTE_NO_TREINO and 'class_weight' not in current_model_params_for_training:
        current_model_params_for_training['class_weight'] = 'balanced'
        print("Usando class_weight='balanced' no modelo (SMOTE não aplicado ao treino e não especificado nos params).")
    elif APLICAR_SMOTE_NO_TREINO and current_model_params_for_training.get('class_weight') == 'balanced':
        print("Aviso: SMOTE foi aplicado, class_weight='balanced' pode ser redundante. Removendo class_weight.")
        if 'class_weight' in current_model_params_for_training:
            del current_model_params_for_training['class_weight']
            
    model = RandomForestClassifier(**current_model_params_for_training)

    print(f"\nTreinando RandomForestClassifier com parâmetros: {current_model_params_for_training}...")
    if current_model_params_for_training.get('verbose', 0) > 0:
        print("(Scikit-learn 'verbose' mostrará o progresso da construção das árvores abaixo)")
    
    start_training_time = time.time()
    try:
        model.fit(X_train_processed, y_train_processed)
        end_training_time = time.time()
        training_duration = end_training_time - start_training_time
        print(f"Modelo treinado com sucesso em {training_duration:.2f} segundos.")
    except Exception as e:
        print(f"Erro durante o treinamento do modelo: {e}")
        model = None 
else:
    print("Dados de treino processados não disponíveis. Pulando treinamento.")

### 4.5 Avaliação do Modelo

Este trecho de código **avalia o desempenho do modelo treinado** no conjunto de teste (`X_test`, `y_test`). Primeiramente, verifica se o modelo e os dados de teste estão disponíveis. Se sim, ele gera previsões de probabilidade, converte-as em classes usando o `LIMIAR_CLASSIFICACAO`, e então exibe um **relatório de classificação** detalhado e uma **matriz de confusão** visual (usando `seaborn` e `matplotlib`). Adicionalmente, calcula e imprime os componentes da matriz de confusão (VN, FP, FN, VP) e a **acurácia geral**, tratando possíveis erros durante o processo.


In [ ]:
if model is not None and X_test is not None and y_test is not None:
    print("\nAvaliando o modelo no conjunto de teste...")
    try:
        proba_predictions = model.predict_proba(X_test)[:, 1]
        print(f"Utilizando limiar de classificação: {LIMIAR_CLASSIFICACAO}")
        predictions = (proba_predictions >= LIMIAR_CLASSIFICACAO).astype(int)

        print("\n--- Relatório de Classificação ---")
        print(classification_report(y_test, predictions, zero_division=0))

        print("\n--- Matriz de Confusão ---")
        cm = confusion_matrix(y_test, predictions)
        
        plt.figure(figsize=(6,4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=['Não Fraude (Prev)', 'Fraude (Prev)'], 
                    yticklabels=['Não Fraude (Real)', 'Fraude (Real)'],
                    annot_kws={"size": 12})
        plt.title('Matriz de Confusão', fontsize=14)
        plt.ylabel('Classe Real', fontsize=12)
        plt.xlabel('Classe Prevista', fontsize=12)
        plt.show()
        
        tn, fp, fn, tp = 0,0,0,0 
        if cm.size == 4: 
            tn, fp, fn, tp = cm.ravel()
        elif cm.size == 1 and len(np.unique(y_test)) == 1 : 
             if y_test.iloc[0] == 0: tn = cm[0,0]
             else: tp = cm[0,0]
        
        print(f"\nDetalhes da Matriz de Confusão:")
        print(f"Verdadeiros Negativos (Não-Fraudes OK): {tn}")
        print(f"Falsos Positivos (Não-Fraudes -> Fraude): {fp} <-- Erro Tipo I")
        print(f"Falsos Negativos (Fraudes -> Não Fraude): {fn} <-- Erro Tipo II (CRÍTICO para fraude)")
        print(f"Verdadeiros Positivos (Fraudes OK): {tp}")

        accuracy = accuracy_score(y_test, predictions)
        print(f"\nAcurácia Geral: {accuracy:.4f}")

    except Exception as e:
        print(f"Erro durante a avaliação ou predição: {e}")
else:
    print("Modelo não treinado ou dados de teste não disponíveis. Pulando avaliação.")

### 4.6 Importância das Features (Random Forest)

Este bloco de código visualiza a **importância das features** conforme determinado pelo modelo treinado (se ele suportar, como o RandomForest). Ele verifica se o modelo existe, se `X_final` é um DataFrame e se o atributo `feature_importances_` está disponível. Em caso afirmativo, extrai as importâncias, associa-as aos nomes das colunas de `X_final`, e exibe as **15 features mais importantes** em uma tabela e em um gráfico de barras horizontal (`seaborn`), facilitando a interpretação de quais variáveis mais influenciaram as previsões do modelo.

In [ ]:
if model is not None and isinstance(X_final, pd.DataFrame) and hasattr(model, 'feature_importances_'):
    print("\n--- Importância das Features (Top 15) ---")
    try:
        importances = model.feature_importances_
        feature_names = X_final.columns 
        
        feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
        feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

        print("Primeiras 15 features mais importantes:")
        display(feature_importance_df.head(15))

        plt.figure(figsize=(10, 8))
        sns.barplot(x='importance', y='feature', data=feature_importance_df.head(15), 
                    palette='viridis', hue='feature', legend=False) # UPDATED LINE
        plt.title('Importância das Features (Top 15)', fontsize=14)
        plt.xlabel('Importância', fontsize=12)
        plt.ylabel('Feature', fontsize=12)
        plt.tight_layout() 
        plt.show()
    except Exception as e:
        print(f"Erro ao calcular/plotar importância das features: {e}")
else:
    print("Modelo não treinado, X_final não é DataFrame ou o modelo não suporta 'feature_importances_'. Pulando.")

## 5. Conclusão e Próximos Passos
Ao longo deste projeto, meu objetivo foi desenvolver um modelo de Machine Learning eficaz para a detecção de fraudes em transações de cartão de crédito, utilizando um dataset realista e altamente desbalanceado.

Performance do Modelo e Descobertas:

Com a configuração atual, que envolveu o uso de dados reais (creditcard.csv), engenharia de features e a aplicação da técnica SMOTE no conjunto de treino para lidar com o desbalanceamento de classes, consegui resultados bastante promissores com o modelo RandomForestClassifier. Nos meus melhores experimentos, utilizando parâmetros como n_estimators=150, max_depth=30, min_samples_leaf=1 e min_samples_split=5 (ou 2), e um limiar de classificação de 0.5, alcancei um F1-score para a classe de fraude (Classe 1) em torno de 0.87.

Impacto do SMOTE: A aplicação do SMOTE no conjunto de treino foi crucial. Antes de sua utilização (ou quando apenas o class_weight='balanced' era usado no modelo), a revocação (recall) para a classe de fraude era significativamente mais baixa. Após o SMOTE, que balanceou artificialmente as classes no conjunto de treino, observei um aumento substancial na revocação – o modelo passou a identificar uma porcentagem muito maior das fraudes reais (por exemplo, de um recall inicial baixo para a faixa de 0.80-0.84). Isso, no entanto, veio com uma ligeira queda na precisão em alguns casos, indicando um aumento nos falsos positivos, o que é um trade-off comum.

Efeito da Engenharia de Features: As features que criei, como Amount_per_Time, Log1p_Amount e as interações cíclicas da feature Time (HourOfDay_sin, HourOfDay_cos), contribuíram para a performance do modelo. A análise de importância das features (feature importances) mostrou que algumas dessas novas features, juntamente com certas features V do dataset original (como V14, V4, V10, V12, V17), foram bastante influentes nas decisões do modelo.

Efeito do Limiar de Classificação: Mantive o limiar de classificação em 0.5 para a maioria dos experimentos reportados. No entanto, reconheço que ajustar este limiar (usando a saída de predict_proba()) é uma próxima etapa importante para otimizar o equilíbrio entre precisão e revocação, dependendo dos custos associados a falsos positivos versus falsos negativos no contexto de negócio.
Considerando o F1-score de 0.87 (com Precisão de 0.90 e Revocação de 0.84 para a classe de fraude), considero que o modelo atingiu um bom nível de performance, identificando 103 das 123 fraudes no conjunto de teste, com apenas 11 alarmes falsos.

Próximos Passos e Melhorias Futuras:

Apesar dos resultados encorajadores, há várias avenidas para exploração e melhoria:

Busca de Hiperparâmetros Mais Exaustiva: Utilizar GridSearchCV ou RandomizedSearchCV com uma grade de parâmetros mais ampla para o RandomForestClassifier (e potencialmente para o SMOTE) poderia refinar ainda mais o modelo. O GridSearchCV que executei com um número limitado de candidatos já confirmou um bom conjunto de parâmetros, mas uma busca mais extensa pode revelar combinações ainda melhores.
Testar Outros Algoritmos: Explorar algoritmos de Gradient Boosting como XGBoost, LightGBM ou CatBoost, que são conhecidos por sua alta performance em datasets tabulares e desbalanceados, e oferecem parâmetros específicos para lidar com o desbalanceamento (ex: scale_pos_weight).

Engenharia de Features Avançada: Dedicar mais tempo à análise exploratória dos dados para identificar outras interações ou transformações de features que possam ser mais discriminativas.
Análise Detalhada de Erros: Investigar os casos de Falsos Positivos e Falsos Negativos para entender se há padrões nesses erros que possam guiar melhorias no pré-processamento ou na escolha do modelo.
Técnicas de Detecção de Anomalias: Como a fraude é um evento raro, abordagens de detecção de anomalias (como Isolation Forest ou One-Class SVM) poderiam ser exploradas como alternativas ou em conjunto com os modelos de classificação.

Limiar de Classificação Otimizado: Realizar uma análise sistemática do impacto de diferentes limiares de classificação na curva Precision-Recall para escolher um limiar que otimize o F1-score ou atenda a um requisito específico de negócio (por exemplo, um recall mínimo desejado).
Considerar Custos de Classificação Incorreta: Em um cenário real, atribuir custos diferentes para Falsos Positivos e Falsos Negativos e otimizar o modelo ou o limiar com base nesses custos seria uma etapa importante.

Em conclusão, este projeto me proporcionou uma experiência prática valiosa na construção de um pipeline de detecção de fraudes, desde a preparação dos dados até a avaliação do modelo, e me deu uma base sólida para futuras explorações e otimizações neste domínio desafiador.